In [ ]:
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers
import nibabel as nib
import os
import pandas as pd
import glob
import platform
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## Paths Input Here
analysis = "roi_roi_overlap/yeo_vta_cross_correl"
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    conn_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/seeds/non_fl_vtas_bilateral_t_con'
    roi_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/seed_generated_networks/yeo_networks_from_thick_yeo_seeds'
    
    # clin_path = 'path to clinical values'
    out_dir = os.path.join(os.path.dirname(conn_path), f'{analysis}')
    
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Mac style')
    print('I will save to:', out_dir)
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    print('add the paths')

if os.path.isdir(out_dir) != True:
    os.makedirs(out_dir)

save = True

In [ ]:
#Get and Prepare Dataframes
from calvin_utils.import_matrices import import_matrices_from_folder
vta_df = import_matrices_from_folder(conn_path, file_pattern='/*.nii')
display(vta_df)

In [ ]:
yeo_df = import_matrices_from_folder(roi_path, file_pattern='/*.nii')
display(yeo_df)

In [ ]:
cross_correlation_diagonal = (vta_df.iloc[:,1]) * (yeo_df.iloc[:,1]).transpose()


In [ ]:
import pandas as pd
from sklearn.decomposition import PCA

# Define the number of principal components to extract from each voxelwise PCA
n_components = 7

# Loop over each column in vta_df and perform cross-correlation with each column in yeo_df
dfs_pca = []

#Iterate over each subject network
for col in vta_df.columns:
    vta_col = vta_df[col].values
    dfs_corr = []
    
    #Iterate over each reference network
    for yeo_col in yeo_df.columns:
        #Achieve the diagonal of the cross-correlation matrix
        cross_correlation_diagonal = (vta_df.iloc[:,1] - np.mean(vta_col)) * (yeo_col - np.mean(yeo_col)).transpose()
        
        
        std_vta_col = np.std(vta_col)
        std_yeo_col = np.std(yeo_col)
        corr = corr / (len(vta_col) * std_vta_col * std_yeo_col)
        dfs_corr.append(pd.DataFrame({'corr': corr}))
    # Concatenate the correlation dataframes along the columns axis
    df_corr = pd.concat(dfs_corr, axis=1)
    # Perform voxelwise PCA on the correlation dataframe
    pca = PCA(n_components=n_components)
    pca.fit(df_corr.values.T)
    # Extract the first principal component and store it in a new dataframe
    df_pca = pd.DataFrame({col: pca.components_[0]})
    dfs_pca.append(df_pca)

# Concatenate the principal component dataframes along the columns axis to create a new dataframe
df_pca_all = pd.concat(dfs_pca, axis=1)
